# 一、赛事背景

当代医学诊断实践中，超声成像技术凭借其非侵入性、无放射性、成本效益高及实时性等显著特点，已广泛成为各类医疗场景中不可或缺的诊断工具。该技术通过发射高频声波并捕捉其回波信号，构建出人体内部结构的精确图像/视频，从而为临床医生提供了关于器官、组织和血管系统的详尽信息。

尽管如此，超声成像的数据解析与分析过程往往高度依赖于专业医生的主观判断和个人经验，这种依赖可能引发诊断结果的不一致性及误差。此外，众多不同的疾病在超声数据上可能展现出相似的特性，这增加了仅凭视觉检查准确区分和识别特定病种的难度，特别是在多病种共存的情况下。 

因此，研究基于超声数据的多病种疾病预测方法显得尤为重要。本次赛事旨在激励全球AI领域研究者开发能够自动处理和分析超声数据，并准确预测患者可能患有的多种疾病的算法和方法。此类技术不仅能提升诊断的准确性和效率，降低误诊或漏诊的风险，而且对优化医疗资源配置、提高诊疗效率、降低医疗成本具有潜在的重大影响。同时，可进一步提升基于超声数据的多病种疾病预测方法的性能，并推动其在实际临床环境中的应用。

# 二、赛事任务

为研究利用超声数据进行多病种的疾病预测算法，本次大赛提供了数据集训练样本，参赛者需根据提供的超声样本构建模型，对数据集进行处理。

# 三、 评审规则

**1.数据说明**

本次大赛实行一轮赛制，所用数据集图像格式为npy。

| 数据类别 | 文件夹名 | 数据格式 | 解释   |
| -------- | -------- | -------- | ------ |
| 基本数据 | test     | npy      | 测试集 |
| 基本数据 | train    | npy      | 训练集 |

**2.评估指标**

本模型依据提交的结果文件，采用macro F1-score进行评价。

**3.评测及排行**

1）比赛均提供下载数据，选手在本地进行算法调试，在比赛页面提交结果。

2）每支团队每天最多提交5次。

3）排行按照得分从高到低排序，排行榜将选择团队的历史最优成绩进行排名。

# 四、作品提交要求

1、文件格式：按照csv格式提交。

2、文件大小：无要求。

3、提交次数限制：每支队伍整个赛期最多30次。

4、文件详细说明：

1) 以csv格式提交，编码为UTF-8，第一行为表头。
2) 提交格式见样例。

5、排行榜更新结束后，前三名选手需要提交代码、模型和说明文档。

# 五、赛程规则

本赛题实行一轮赛制

**赛程周期**

8月19日-9月23日

1、8月19日10：00发布训练集、测试集；

2、比赛作品提交截止日期为9月23日17：00，公布名次日期为9月下旬。

**现场答辩**

1、最终前三名团队将受邀参加科大讯飞AI开发者大赛年度总决赛并于现场进行答辩；

2、答辩以（10mins陈述+5mins问答）的形式进行；

3、根据作品成绩和答辩成绩综合评分（作品成绩占比70％，现场答辩份数占比30％）。

# 六、奖项设置

本赛题设立一、二、三等奖共三名，具体详情如下：

**【奖项激励】**

1、TOP3团队颁发获奖证书

2、赛道奖金，第一名5000元、第二名3000元、第三名2000元

**【资源激励】**

1、讯飞开放平台优质AI能力个人资源包

2、讯飞AI全链创业扶持资源

3、讯飞绿色实习/就业通道

注：

1.鼓励选手分享参赛心得、参赛技术攻略、大赛相关技术或产品使用体验等文章至组委会邮箱（AICompetition@iflytek.com），有机会获得大赛周边；

2.赛事规则及奖金发放解释权归科大讯飞所有；以上全部奖金均为税前金额，将由主办方代扣代缴个人所得税。

In [1]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

%pylab inline

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
train_path = glob.glob('./train/*/*.npy')
np.random.shuffle(train_path)

labels = ['Anomalies', 'Cyst', 'Inflammation', 'Tumor', 'Vascular']
train_label = [labels.index(x.split('/')[-2]) for x in train_path]

In [3]:
DATA_CACHE = {}
class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = np.load(self.img_path[index])
            DATA_CACHE[self.img_path[index]] = img
        if self.transform is not None:
            img = self.transform(image = img)['image']
        img = img.transpose([2,0,1])
        return img, torch.from_numpy(np.array(self.img_label[index]))
    def __len__(self):
        return len(self.img_path)

In [4]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet18(True)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 5)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

In [5]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Train loss', loss.item())
            
        train_loss += loss.item()
    
    return train_loss/len(train_loader)
            
def validate(val_loader, model, criterion):
    model.eval()
    
    val_acc = 0.0
    
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            val_acc += (output.argmax(1) == target).sum().item()
            
    return val_acc / len(val_loader.dataset)

def predict(test_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    
    test_pred = []
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(test_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)

In [6]:
import albumentations as A
train_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[:-100], train_label[:-100],
            A.Compose([
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    ), batch_size=30, shuffle=True, num_workers=1, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[-100:], train_label[-100:],
            A.Compose([
            # A.HorizontalFlip(p=0.5),
            # A.RandomContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    ), batch_size=30, shuffle=False, num_workers=1, pin_memory=False
)

In [7]:
model = XunFeiNet()
model = model.to('cuda')
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), 0.001)

for _  in range(8):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc  = validate(val_loader, model, criterion)
    train_acc = validate(train_loader, model, criterion)
    print(train_loss, train_acc, val_acc)

/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train loss 1.6971927881240845
0.7282405907909075 0.655793025871766 0.71
Train loss 0.16219213604927063
0.19310481796662013 0.9617547806524185 0.95
Train loss 0.07772817462682724
0.08410332364340624 0.8807649043869517 0.85
Train loss 0.166463702917099
0.23089910261332988 0.7851518560179978 0.72
Train loss 0.09972250461578369
0.13735906984657048 0.8177727784026997 0.83
Train loss 0.08735839277505875
0.06372769172303379 0.9853768278965129 0.99
Train loss 0.062059804797172546
0.024383619807971022 1.0 0.99
Train loss 0.009021206758916378
0.013137612421996892 0.9988751406074241 0.99


In [11]:
model.eval()
test_path = glob.glob('./test/*.npy')
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path, [0] * len(test_path),
            A.Compose([
            # A.HorizontalFlip(p=0.5),
            # A.RandomContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    ), batch_size=30, shuffle=False, num_workers=1, pin_memory=False
)
pred = predict(test_loader, model, criterion)

In [19]:
pd.DataFrame({
    'uuid': [x.split('/')[-1][:-4] for x in test_path],
    'label': [labels[x] for x in pred.argmax(1)]
}).to_csv('submit.csv', index=None)